In [ ]:
%pip install numpy pandas matplotlib scikit-learn h5py

In [ ]:
# import h5py
import scipy.io
import scipy.stats
import sklearn.metrics
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
featFileName = "/Users/guancong/Desktop/sch/3110_ITP/BatteryLifeTimeML/Data/features_combined.csv"
batch_pd = pd.read_csv(featFileName, index_col=False)
batch_pd

In [ ]:
# feat = batch_pd.drop(['policy','barcode','cycle_life','QDiffLinVar'], axis=1).to_numpy()
feat = batch_pd['QDiffLinVar'].to_numpy()
# feat = np.sort(feat)
# feat = np.flip(feat)
feat

In [ ]:
bat_info = batch_pd.loc[:,'policy':'cycle_life']

## Features

In [ ]:
plt.scatter(feat, bat_info["cycle_life"])

plt.show()

## Model Building

In [ ]:
# randomly partition into train and test
# feat_train, feat_test, label_train, label_test = train_test_split(
#     feat, np.log10(bat_info["cycle_life"]) ,
#     test_size=0.5
#     # the paper split every other cell, resulting in 50-50 mix
# )
feat_train = np.reshape(feat[0::2], (-1,1))
feat_test = np.reshape(feat[1::2], (-1,1))
label_train = bat_info["cycle_life"][0::2]
label_test = bat_info["cycle_life"][1::2]

In [ ]:
# scale data
feat_train_scaled = scipy.stats.zscore(feat_train)
feat_test_scaled = scipy.stats.zmap(feat_test, feat_train)

In [ ]:
# train model
# Linear Elastic Net with Four-Fold Cross-Validation, alpha = 0.9
# B = linear_model.ElasticNetCV(cv=4, l1_ratio=0.9)

B = linear_model.LinearRegression()
B.fit(feat_train_scaled, label_train)

In [ ]:
# make predictions
ypred_train = B.predict(feat_train_scaled)
ypred_test = B.predict(feat_test_scaled)

In [ ]:
ypred_train

In [ ]:
label_train.to_numpy()

## Evaluation

In [ ]:
# calculate prediction interval (incomplete)
print(sklearn.metrics.mean_squared_error(label_train, ypred_train, squared=False))
print(sklearn.metrics.mean_squared_error(label_test, ypred_test, squared=False))

In [ ]:
B.score(np.reshape(feat_train_scaled, (-1, 1)), label_train)

In [ ]:
B.score(np.reshape(feat_test_scaled, (-1,1)), label_test)

In [ ]:
plt.plot(np.arange(2000),np.arange(2000))
plt.scatter(label_train, ypred_train, label="train", marker="o")
plt.scatter(label_test, ypred_test, label="test", marker="^")
plt.legend()
plt.show()

In [ ]:
# plt.plot(np.arange(2000),np.arange(2000))
plt.scatter(feat_train_scaled[:,0], np.log10(label_train), label="train", marker="o")
plt.scatter(feat_test_scaled[:,0], np.log10(label_test), label="test", marker="^")
sorting_order = np.argsort(feat_train_scaled[:,0])
plt.plot(feat_train_scaled[:,0][sorting_order], np.log10(ypred_train[sorting_order]))
plt.legend()
plt.show()